In [1]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# set your file dir path
npy_directory = '/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Conch/TCGA-*/'
feature1_files = glob.glob(os.path.join(npy_directory, "*_0_1024.npy"))
feature2_files = glob.glob(os.path.join(npy_directory, "*_1_512.npy"))
feature3_files = glob.glob(os.path.join(npy_directory, "*_1_1024.npy"))

df_fea1 = pd.DataFrame(feature1_files, columns=['fea1_file_path'])
df_fea2 = pd.DataFrame(feature2_files, columns=['fea2_file_path'])
df_fea3 = pd.DataFrame(feature3_files, columns=['fea3_file_path'])

patients_file = '/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/TCGA_Clinical.tsv'
patients_list = pd.read_csv(patients_file, sep='\t')[['case_submitter_id','project_id']].drop_duplicates()
patients_list.columns = ['patient_id','project']

In [3]:
df_fea1['slide_id'] = df_fea1['fea1_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea2['slide_id'] = df_fea2['fea2_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])
df_fea3['slide_id'] = df_fea3['fea3_file_path'].apply(lambda x: os.path.basename(x).split('.')[0])

df_fea1 = df_fea1.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)
df_fea2 = df_fea2.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)
df_fea3 = df_fea3.drop_duplicates(subset='slide_id', keep='first').reset_index(drop=True)

df_fea = pd.merge(df_fea1, df_fea2, on='slide_id', how='inner')
df_fea = pd.merge(df_fea, df_fea3, on='slide_id', how='inner')

df_fea['patient_id'] = df_fea['slide_id'].apply(lambda x: x[:12])

df = pd.merge(df_fea, patients_list, on='patient_id', how='inner')

In [4]:
df.head()

,fea1_file_path,slide_id,fea2_file_path,fea3_file_path,patient_id,project
0,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-FK-A4UB-01Z-00-DX1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-FK-A4UB,TCGA-THCA
1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-EM-A22O-01Z-00-DX1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-EM-A22O,TCGA-THCA
2,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-FK-A3SE-01Z-00-DX1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-FK-A3SE,TCGA-THCA
3,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-DE-A4MD-01Z-00-DX1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-DE-A4MD,TCGA-THCA
4,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-L6-A4EP-01Z-00-DX1,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Con...,TCGA-L6-A4EP,TCGA-THCA


In [5]:
# 定义聚类中心计算函数
def get_cluster_centers_indices(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(data)
    labels = kmeans.labels_
    
    cluster_centers_indices = []
    for i in range(n_clusters):
        cluster_indices = np.where(labels == i)[0]
        center_index = cluster_indices[np.argmin(np.linalg.norm(data[cluster_indices] - kmeans.cluster_centers_[i], axis=1))]
        cluster_centers_indices.append(center_index)
    
    return cluster_centers_indices

# 定义处理单行数据的函数
def process_row(i, df, n_clusters):
    feature1_content = np.load(df.iloc[i]['fea1_file_path'], allow_pickle=True)
    feature1 = feature1_content[()]['feature'].cpu().numpy()
    feature1_cor = feature1_content[()]['index']
    
    feature2_content = np.load(df.iloc[i]['fea2_file_path'], allow_pickle=True)
    feature2 = feature2_content[()]['feature'].cpu().numpy()
    feature2_cor = feature2_content[()]['index']
    
    feature3_content = np.load(df.iloc[i]['fea3_file_path'], allow_pickle=True)
    feature3 = feature3_content[()]['feature'].cpu().numpy()
    feature3_cor = feature3_content[()]['index']
    
    try:
        f1_cc = get_cluster_centers_indices(feature1, n_clusters[0])
        f1_cc_cor = [feature1_cor[index] for index in f1_cc]
    except Exception:
        f1_cc_cor = []
    try:
        f2_cc = get_cluster_centers_indices(feature2, n_clusters[1])
        f2_cc_cor = [feature2_cor[index] for index in f2_cc]
    except Exception:
        f2_cc_cor = []
    try:
        f3_cc = get_cluster_centers_indices(feature3, n_clusters[2])
        f3_cc_cor = [feature3_cor[index] for index in f3_cc]
    except Exception:
        f3_cc_cor = []
    
    return {'f1024_cc': ','.join(f1_cc_cor), 
            'f2048_cc': ','.join(f2_cc_cor), 
            'f4096_cc': ','.join(f3_cc_cor),
            'slide_id': df['slide_id'].iloc[i],
            'project': df['project'].iloc[i]}

def process_data_multithreaded(df, n_clusters, max_threads=4):
    data = []
    
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(process_row, i, df, n_clusters) for i in range(df.shape[0])]
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            data.append(future.result())
    
    return data

In [6]:
n_clusters = [16, 8, 4]

for project in df['project'].unique():
    print(project)
    df_sub = df[df['project'] == project]
    print(df_sub.shape)
    processed_data = process_data_multithreaded(df_sub, n_clusters, max_threads=5)
    processed_data = pd.DataFrame(processed_data)
    processed_data.to_csv(os.path.join('/bask/homes/a/asiw9691/PathVLM/WSI_Dataset/Conch_CC', '{}_cc.csv'.format(project)), index=False)

TCGA-THCA
(518, 6)


100%|██████████| 600/600 [01:30<00:00,  6.63it/s]


TCGA-LUAD
(530, 6)


100%|██████████| 530/530 [01:02<00:00,  8.41it/s]


TCGA-LUSC
(512, 6)


...

100%|██████████| 157/157 [00:22<00:00,  7.07it/s]


TCGA-UVM
(80, 6)


100%|██████████| 80/80 [00:07<00:00, 10.93it/s]


TCGA-DLBC
(44, 6)


100%|██████████| 44/44 [00:04<00:00, 10.53it/s]


In [7]:
processed_data

,f1024_cc,f2048_cc,f4096_cc,slide_id,project
0,"17409_30721_1024.png,11265_22529_1024.png,4198...","8193_14337_2048.png,6145_8193_2048.png,24577_1...","24577_8193_4096.png,12289_24577_4096.png,4097_...",TCGA-FF-8041-01Z-00-DX1,TCGA-DLBC
1,"39937_79873_1024.png,49153_46081_1024.png,2355...","26625_71681_2048.png,63489_63489_2048.png,4505...","12289_12289_4096.png,45057_61441_4096.png,2457...",TCGA-GS-A9TZ-01Z-00-DX1,TCGA-DLBC
2,"69633_59393_1024.png,99329_77825_1024.png,1136...","63489_47105_2048.png,126977_32769_2048.png,983...","163841_36865_4096.png,36865_45057_4096.png,614...",TCGA-G8-6326-01Z-00-DX1,TCGA-DLBC
3,"13313_33793_1024.png,12289_31745_1024.png,5017...","12289_32769_2048.png,12289_30721_2048.png,1228...",,TCGA-FF-8062-01Z-00-DX1,TCGA-DLBC
4,"60417_12289_1024.png,9217_27649_1024.png,28673...","24577_6145_2048.png,8193_24577_2048.png,55297_...","69633_28673_4096.png,86017_16385_4096.png,8193...",TCGA-RQ-AAAT-01Z-00-DX1,TCGA-DLBC
5,"133121_31745_1024.png,51201_60417_1024.png,113...","143361_55297_2048.png,139265_20481_2048.png,13...","32769_32769_4096.png,131073_61441_4096.png,147...",TCGA-G8-6325-01Z-00-DX1,TCGA-DLBC
6,"74753_76801_1024.png,102401_54273_1024.png,460...","71681_65537_2048.png,69633_30721_2048.png,1024...","65537_73729_4096.png,32769_40961_4096.png,4097...",TCGA-FA-A4XK-01Z-00-DX1,TCGA-DLBC
7,"21505_27649_1024.png,99329_19457_1024.png,9318...","114689_26625_2048.png,12289_24577_2048.png,921...","20481_16385_4096.png,98305_20481_4096.png,1228...",TCGA-GS-A9TV-01Z-00-DX1,TCGA-DLBC
8,"51201_8193_1024.png,86017_51201_1024.png,78849...","51201_22529_2048.png,6145_14337_2048.png,83969...","32769_16385_4096.png,69633_4097_4096.png,57345...",TCGA-FF-8042-01Z-00-DX1,TCGA-DLBC
9,"12289_29697_1024.png,99329_30721_1024.png,8601...","92161_40961_2048.png,14337_57345_2048.png,1003...","90113_49153_4096.png,8193_57345_4096.png,86017...",TCGA-G8-6907-01Z-00-DX1,TCGA-DLBC
